In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Stroke/GSE117064'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Serum miRNA-based risk prediction for stroke"
!Series_summary	"The serum miRNAs were correlated with predicted the incidence of stroke."
!Series_overall_design	"Serum microRNA profiles of 1785 samples, which consist of 173 of CVD patients, 1612 of non-CVD control."
Sample Characteristics Dictionary:
{0: ['tissue: Serum'], 1: ['Sex: Female', 'Sex: Male'], 2: ['age: 50', 'age: 47', 'age: 43', 'age: 58', 'age: 41', 'age: 56', 'age: 44', 'age: 40', 'age: 54', 'age: 49', 'age: 63', 'age: 46', 'age: 66', 'age: 51', 'age: 42', 'age: 72', 'age: 57', 'age: 52', 'age: 53', 'age: 45', 'age: 48', 'age: 61', 'age: 73', 'age: 62', 'age: 75', 'age: 68', 'age: 65', 'age: 81', 'age: 79', 'age: 69'], 3: ['bmi: 21.2', 'bmi: 22.4', 'bmi: 23.3', 'bmi: 20', 'bmi: 20.8', 'bmi: 20.2', 'bmi: 17.8', 'bmi: 22.9', 'bmi: 26.2', 'bmi: 31.4', 'bmi: 22.3', 'bmi: 24.6', 'bmi: 19', 'bmi: 18', 'bmi: 20.3', 'bmi: 21.3', 'bmi: 25.6', 'bmi: 24.2', 'bmi: 34.6', 'bmi: 23.6', 'bmi: 18.6'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = False  # As per the series title and summary, the data is focused on miRNA, not gene expression

# 2. Variable Availability and Data Type Conversion
# Analyzing the sample characteristics dictionary

# Checking for 'Stroke' data availability
# The data focuses on CVD (cardiovascular diseases) patients and controls but doesn't explicitly list stroke-related data
trait_row = None

# Checking for 'age' data availability
# Age data found in key 2
age_row = 2

# Checking for 'gender' data availability
# Gender data found in key 1
gender_row = 1

# Data Type Conversion Functions
def convert_trait(value):
    return None  # No conversion needed since trait data is not available

def convert_age(value):
    try:
        return int(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(':')[1].strip().lower()
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
        else:
            return None
    except:
        return None

# 3. Save Metadata
save_cohort_info('GSE117064', './preprocessed/Stroke/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Stroke', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Stroke/trait_data/GSE117064.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
